## Integrantes do grupo:

**(Nome do aluno) - (R.A)**

Luiz Fernando Ferreira Berber - 817123421

Lucas Alves de Sousa - 817116864

Matheus Henrique Silva Paiva - 817123241 

Mateus Sampaio de Souza - 817123420

Pedro Jacó Cunha Mesquita - 817124121

In [1]:
!pip install numpy
!pip install pandas
!pip install regex
!pip install mysql-connector-python

In [2]:
import numpy as np
import pandas as pd
import mysql.connector
import regex

from mysql.connector import Error
from pathlib import Path

In [3]:
def get_files_from_path(filepath):
    path = Path(filepath)
    files = []

    for childs in path.iterdir():
        if childs.is_file(): files.append(childs.parts[len(childs.parts) - 1])

    return files

In [4]:
def create_connection(host_name, user_name, user_password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [5]:
def execute_query(connection, query):
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
        connection.commit()
    except Error as e:
        print(f"The error '{e}' occurred")
        print(f'Query with error: {query}')

In [6]:
def insert_dimensao_orgao(dados_despesa, connection):
    dimensao_orgao = pd.DataFrame({
        'cod_orgao_superior':dados_despesa['Código Órgão Superior'],
        'nome_orgao_superior':dados_despesa['Nome Órgão Superior'],
        'cod_orgao_subordinado':dados_despesa['Código Órgão Subordinado'],
        'nome_orgao_subordinado':dados_despesa['Nome Órgão Subordinado'],
        'cod_unidade_orcamentaria':dados_despesa['Código Unidade Orçamentária'],
        'nome_unidade_orcamentaria':dados_despesa['Nome Unidade Orçamentária']
    })

    for i in range(dimensao_orgao.shape[0]):
        insert_dimensao_orgao = f"""
        INSERT INTO orcamento_publico.TD_ORGAO
        (
            COD_ORGAO_SUPERIOR
        ,	NOME_ORGAO_SUPERIOR
        ,	COD_ORGAO_SUBORDINADO
        ,	NOME_ORGAO_SUBORDINADO
        ,	COD_UNIDADE_ORCAMENTARIA
        ,	NOME_UNIDADE_ORCAMENTARIA
        ,	DATA_CRIACAO
        )
        VALUES
        (
            {dimensao_orgao.loc[i]['cod_orgao_superior']}
        ,   '{dimensao_orgao.loc[i]['nome_orgao_superior']}'
        ,   {dimensao_orgao.loc[i]['cod_orgao_subordinado']}
        ,   '{dimensao_orgao.loc[i]['nome_orgao_subordinado']}'
        ,   {dimensao_orgao.loc[i]['cod_unidade_orcamentaria']}
        ,   '{dimensao_orgao.loc[i]['nome_unidade_orcamentaria']}'
        ,   NOW()
        );"""

        execute_query(connection, insert_dimensao_orgao)

In [7]:
def insert_dimensao_programa(dados_despesa, connection):
    dimensao_programa = pd.DataFrame({
        'cod_programa_orcamentario':dados_despesa['Código Programa Orçamentário'],
        'nome_programa_orcamentario':dados_despesa['Nome Programa Orçamentário'],
        'cod_acao':dados_despesa['Código Ação'],
        'nome_acao':dados_despesa['Nome Ação']
    })
    
    for i in range(dimensao_programa.shape[0]):
        insert_dimensao_programa = f"""
        INSERT INTO ORCAMENTO_PUBLICO.TD_PROGRAMA(Cod_Programa_Orcamentario, Nome_Programa_Orcamentario, Cod_Acao, Nome_Acao, Data_Criacao)
        VALUES(
            {dimensao_programa.loc[i]['cod_programa_orcamentario']}
            , '{dimensao_programa.loc[i]['nome_programa_orcamentario']}'
            , '{dimensao_programa.loc[i]['cod_acao']}'
            , '{dimensao_programa.loc[i]['nome_acao']}'
            , NOW()
        );
        """
        
        execute_query(connection, insert_dimensao_programa)

In [8]:
def insert_dimensao_data(mes, ano, connection):
    insert_dimensao_data = f"""
    INSERT INTO orcamento_publico.TD_DATA
    (
        MES
    ,	ANO
    ,	DATA_CRIACAO
    )
    VALUES
    (
        {mes}
    ,   {ano}
    ,   NOW()
    );
    """
    
    execute_query(connection, insert_dimensao_data)

In [9]:
def insert_dimensao_area_atuacao(dados_orcamento_despesa, connection):
    dados_orcamento = pd.DataFrame({
        'cod_funcao':dados_orcamento_despesa['CÓDIGO FUNÇÃO'],
        'nome_funcao':dados_orcamento_despesa['NOME FUNÇÃO'],
        'cod_sub_funcao':dados_orcamento_despesa['CÓDIGO SUBFUNÇÃO'],
        'nome_sub_funcao':dados_orcamento_despesa['NOME SUBFUNÇÃO']
    })
    
    for i in range(dados_orcamento.shape[0]):
        insert_dimensao_area_atuacao = f"""
        INSERT INTO orcamento_publico.TD_AREA_ATUACAO
        (
            COD_FUNCAO
        ,	NOME_FUNCAO
        ,	COD_SUB_FUNCAO
        ,	NOME_SUB_FUNCAO
        ,	DATA_CRIACAO
        )
        VALUES
        (
         {dados_orcamento.loc[i]['cod_funcao']},
         '{dados_orcamento.loc[i]['nome_funcao']}',
         {dados_orcamento.loc[i]['cod_sub_funcao']},
         '{dados_orcamento.loc[i]['nome_sub_funcao']}',
         NOW()
        );
        """

        execute_query(connection, insert_dimensao_area_atuacao)

In [10]:
mysql_host_name = '127.0.0.1'
mysql_user_name = 'mateus'
mysql_password = 'root123'

In [11]:
connection = create_connection(mysql_host_name, mysql_user_name, mysql_password)

Connection to MySQL DB successful


In [12]:
path_dados = '../Datas'

In [13]:
path_despesa = f'{path_dados}/Despesas'

arquivos = get_files_from_path(path_despesa)

In [14]:
for arquivo in arquivos:
    dados_despesa = pd.read_csv(f'{path_despesa}/{arquivo}', sep=';', encoding='mbcs')
    
    match = regex.match('(?P<ano>\d{4})(?P<mes>\d{2})', arquivo)
    ano = match.group('ano')
    mes = match.group('mes')
    
    insert_dimensao_data(mes, ano, connection)
    insert_dimensao_orgao(dados_despesa, connection)
    insert_dimensao_programa(dados_despesa, connection)

The error '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SANEAMENTO EM ESCOLAS''
            , NOW()
        )' at line 6' occurred
Query with error: 
        INSERT INTO ORCAMENTO_PUBLICO.TD_PROGRAMA(Cod_Programa_Orcamentario, Nome_Programa_Orcamentario, Cod_Acao, Nome_Acao, Data_Criacao)
        VALUES(
            1287
            , 'SANEAMENTO RURAL'
            , '10GC'
            , 'IMPLANTAÇÃO E MELHORIA DE SERVIÇOS DE SANEAMENTO EM ESCOLAS PÚBLICAS RURAIS - 'SANEAMENTO EM ESCOLAS''
            , NOW()
        );
        


c:\program files (x86)\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


The error '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SANEAMENTO EM ESCOLAS''
            , NOW()
        )' at line 6' occurred
Query with error: 
        INSERT INTO ORCAMENTO_PUBLICO.TD_PROGRAMA(Cod_Programa_Orcamentario, Nome_Programa_Orcamentario, Cod_Acao, Nome_Acao, Data_Criacao)
        VALUES(
            1287
            , 'SANEAMENTO RURAL'
            , '10GC'
            , 'IMPLANTAÇÃO E MELHORIA DE SERVIÇOS DE SANEAMENTO EM ESCOLAS PÚBLICAS RURAIS - 'SANEAMENTO EM ESCOLAS''
            , NOW()
        );
        
The error '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SANEAMENTO EM ESCOLAS''
            , NOW()
        )' at line 6' occurred
Query with error: 
        INSERT INTO ORCAMENTO_PUBLICO.TD_PROGRAMA(Cod_Programa_Orcamentario, Nome_Programa_Orcamentario, Co

In [15]:
path_orcamento_despesas = f'{path_dados}/OrcamentoDespesas'

arquivos = get_files_from_path(path_orcamento_despesas)

In [16]:
for arquivo in arquivos:
    dados_orcamento_despesa = pd.read_csv(f'{path_orcamento_despesas}/{arquivo}', sep=';', encoding='mbcs')
    
    insert_dimensao_area_atuacao(dados_orcamento_despesa, connection)

(26445, 24)
